In [1]:
%load_ext autoreload
%autoreload 2

from epoxy import *
import numpy as np
from sklearn.metrics import pairwise
from helpers import *

## Baseline: FlyingSquid

First, we load data:

In [2]:
L_train_orig = np.load('data/L_train.npy')
L_valid_orig = np.load('data/L_valid.npy')
L_test_orig = np.load('data/L_test.npy')

Y_valid = np.load('data/Y_valid.npy')
Y_test = np.load('data/Y_test.npy')

Let's first see what happens if we train a FlyingSquid model without extending the labeling functions. We'll show numbers on the test set in this notebook, but in our paper we tuned on on the validation set.

In [3]:
label_model = train_fs_model_spam(L_train_orig)
evaluate_fs_model_spam(label_model, L_test_orig, Y_test)

Acc: 0.8360	Pre: 0.8667	Rec: 0.7712	F1: 0.8161


As you can see, precision is pretty decent, but recall lags behind.

## Improving Recall with Embeddings

First, let's load up embeddings:

In [4]:
embeddings_train = np.load('data/embeddings_train.npy')
embeddings_valid = np.load('data/embeddings_valid.npy')
embeddings_test = np.load('data/embeddings_test.npy')

First, we'll do some preprocessing -- compute similarity scores between points, and finding the closest point to abstaining points.

In [5]:
# pre-processing

train_to_train_sims = pairwise.cosine_similarity(embeddings_train, embeddings_train)
valid_to_train_sims = pairwise.cosine_similarity(embeddings_valid, embeddings_train)
test_to_train_sims = pairwise.cosine_similarity(embeddings_test, embeddings_train)

mat_abstains_train, closest_pos_train, closest_neg_train = preprocess_lfs(
    L_train_orig, L_train_orig, train_to_train_sims
)
mat_abstains_test, closest_pos_test, closest_neg_test = preprocess_lfs(
    L_train_orig, L_test_orig, test_to_train_sims
)

Now, we'll extend the train and test matrices (note that the thresholds were tuned on the validation set -- we are eliding that step for simplicity in this notebook).

In [6]:
# these thresholds were tuned on the validation set, see paper for details
thresholds = [0.85, 0.85, 0.85, 0.85, 0.81, 0.85, 0.85, 0.88, 0.85, 0.85]

L_train_extended = extend_lfs(
    L_train_orig,
    mat_abstains_train, closest_pos_train, closest_neg_train,
    thresholds
)
L_test_extended = extend_lfs(
    L_test_orig,
    mat_abstains_test, closest_pos_test, closest_neg_test,
    thresholds
)

And now let's evaluate training with the extended labeling functions:

In [7]:
label_model_extended = train_fs_model_spam(L_train_extended)
evaluate_fs_model_spam(label_model_extended, L_test_extended, Y_test)

Acc: 0.8960	Pre: 0.8898	Rec: 0.8898	F1: 0.8898


As you can see, we have improved recall by more than ten points -- virtually for free!

## Training a downstream end model

And if you'd like, you can still use the label model to generate probabilistic labels for a downstream end model, which gives further improvement (see paper for details):

In [8]:
training_labels = label_model_extended.predict_proba_marginalized(L_train_extended)